<a href="https://colab.research.google.com/github/datamaunz/tradeoff-detection/blob/main/Tradeoff_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [ ]:
!pip install sentence-transformers
!pip install kaleido
!pip install plotly
!python -m spacy download en_core_web_sm
!pip install xlsxwriter

## Make sure to use a GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

## Imports

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer, util
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import time
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from xlsxwriter import Workbook

from plotly.subplots import make_subplots

## Load models

In [ ]:
nlp = spacy.load("en_core_web_sm")
modelName = "paraphrase-distilroberta-base-v1"
model = SentenceTransformer(modelName)

## Functions

### Label identification

In [3]:
#### Functions for label identification

def add_stopwords(stopwords_to_be_added):
  
  for stopword in stopwords_to_be_added:
    STOP_WORDS.add(stopword)
  for word in STOP_WORDS:
      lexeme = nlp.vocab[word]
      lexeme.is_stop = True

def sample_comments_and_concatenate_as_string(df, column, number_samples, sep, random_seed):
  sample = df[column].sample(number_samples, random_state=random_seed).dropna().to_list()
  sample_string = f"{sep}".join(sample)
  return sample_string

def get_concatenated_preprocessed_sampled_strings(df, number_of_sampled_comments, random_seed):

  start = time.time()
  cons_items_string = sample_comments_and_concatenate_as_string(df, "cons", number_of_sampled_comments, "\n\n", random_seed)
  pros_items_string = sample_comments_and_concatenate_as_string(df, "pros", number_of_sampled_comments, "\n\n", random_seed)

  cons_string_doc = nlp(cons_items_string)
  pros_string_doc = nlp(pros_items_string)

  end = time.time()
  print(end - start)

  return cons_string_doc, pros_string_doc

def get_list_of_noun_chunks(cons_string_doc, pros_string_doc):

  cons_string_chunks = [x.lemma_ for x in cons_string_doc.noun_chunks]
  pros_string_chunks = [x.lemma_ for x in pros_string_doc.noun_chunks]

  chunks = cons_string_chunks + pros_string_chunks
  return chunks

def chunks_to_docs(chunks):

  start = time.time()
  chunks_docs = [nlp(chunk) for chunk in chunks]
  end = time.time()
  print(end - start)
  return chunks_docs

def present_nouns_in_df(chunks_docs):

  cleaned_chunks = [" ".join([token.lemma_.lower() for token in doc if (token.pos_ not in ["ADJ"]) & (token.is_stop == False) & (token.is_punct == False)]) for doc in chunks_docs]
  cleaned_chunks = [x for x in cleaned_chunks if x != ""]
  df = pd.DataFrame(Counter(cleaned_chunks), index=[0]).T.sort_values(0, ascending=False).reset_index().rename(columns={"index":"lemma", 0:"COUNT"})
  return df

def get_most_frequent_nouns_and_their_similarities(noun_df, number_of_most_frequent_words):

  frequent_words = noun_df.lemma.iloc[:number_of_most_frequent_words].to_list()
  embeddings_frequent_words = model.encode(frequent_words,convert_to_tensor=True, device=0)
  cos_sim = util.pytorch_cos_sim(embeddings_frequent_words, embeddings_frequent_words)  
  return cos_sim, frequent_words

def plot_dendrogram_of_label_similarities(cos_sim, frequent_words):

  number_of_labels = len(frequent_words)
  plt.rcParams['lines.linewidth'] = 5

  correlations = pd.DataFrame(cos_sim).applymap(lambda x: x.cpu().numpy())
  correlations.columns = frequent_words
  dissimilarity = 1 - abs(correlations)
  np.fill_diagonal(dissimilarity.values, 0)
  Z = linkage(squareform(dissimilarity), 'complete')

  labels = correlations.columns

  fig = plt.figure(figsize=(40,number_of_labels * 2))

  ax = fig.add_subplot(1, 1, 1)
  fig.subplots_adjust(bottom = 0.5)
  dendrogram(Z, ax=ax, 
            orientation="left", 
            labels=labels, color_threshold=0.25
            )
  ax.tick_params(axis='x', which='major', labelsize=35)
  ax.tick_params(axis='y', which='major', labelsize=35, pad=20, right=100)

def map_labels_by_similarity_threshold(cos_sim, noun_df, frequent_words, dissimilarity_threshold):

  correlations = pd.DataFrame(cos_sim).applymap(lambda x: x.cpu().numpy())

  dissimilarity = 1 - abs(correlations)
  np.fill_diagonal(dissimilarity.values, 0)
  Z = linkage(squareform(dissimilarity), 'complete')
  labels_clust = fcluster(Z, dissimilarity_threshold, criterion='distance')
  mapping = np.column_stack([frequent_words, labels_clust])
  columns_df = ['lemma', 'cluster_label']
  dfmapping = pd.DataFrame(mapping, columns=columns_df)
  dfmapping = pd.merge(dfmapping, noun_df, left_on="lemma", right_on="lemma")

  return dfmapping

def reduce_similar_labels_top_most_frequent(dfmapping):

  lemmata = []
  cluster_labels = []
  for cluster_label in dfmapping.cluster_label.unique():
    sub_df = dfmapping[dfmapping.cluster_label == cluster_label]
    
    lemma_1 = sub_df.sort_values("COUNT", ascending=False).iloc[0].lemma
    lemma_2 = sub_df.sort_values("COUNT", ascending=False).iloc[-1].lemma

    # use the label with more examples unless it is contained in another label

    if len(dfmapping[dfmapping.lemma.str.contains(lemma_1)]) > len(dfmapping[dfmapping.lemma.str.contains(lemma_2)]):
      lemmata.append(lemma_2)
    else:
      lemmata.append(lemma_1)

    cluster_labels.append(cluster_label)

  label_df = pd.DataFrame(
      {
          "lemma":lemmata,
      "cluster_label":cluster_labels,
      }
  )

  return label_df

def get_most_representative_labels(df, number_of_sampled_comments, number_of_most_frequent_words, labels_to_be_explicitly_ignored,dissimilarity_threshold, stopwords_to_be_added, random_seed, use_similarity_reduction=True, show_similarity_dendrogram=False):

  cons_string_doc, pros_string_doc = get_concatenated_preprocessed_sampled_strings(df, number_of_sampled_comments, random_seed)
  chunks = get_list_of_noun_chunks(cons_string_doc, pros_string_doc)
  add_stopwords(stopwords_to_be_added)
  chunks_docs = chunks_to_docs(chunks)
  noun_df = present_nouns_in_df(chunks_docs)
  noun_df = noun_df[noun_df.lemma.isin(labels_to_be_explicitly_ignored) == False]
  cos_sim, frequent_words = get_most_frequent_nouns_and_their_similarities(noun_df, number_of_most_frequent_words)
  if show_similarity_dendrogram == True: 
    plot_dendrogram_of_label_similarities(cos_sim, frequent_words)
  if use_similarity_reduction == True:
    dfmapping = map_labels_by_similarity_threshold(cos_sim, noun_df, frequent_words, dissimilarity_threshold)
    label_df = reduce_similar_labels_top_most_frequent(dfmapping)
    labels = label_df.lemma.to_list()
  else:
    labels = frequent_words
  return labels, noun_df

### Identify pro-con pairs

In [4]:
def create_similarity_frame(embeddings_comments, embeddings_labels, labels, comments, reviewIDs, reviewType):
  cos_sim = util.pytorch_cos_sim(embeddings_comments, embeddings_labels)  
  df = pd.DataFrame(cos_sim, columns=labels)
  df = df.applymap(lambda x: x.cpu().numpy())
  df[f"{reviewType}_label"] = df.idxmax(axis=1)
  df[f"{reviewType}_score"] = df.max(axis=1)
  df[f'{reviewType}_comment'] = comments
  df["reviewID"] = reviewIDs
  df = df.sort_values(f"{reviewType}_score", ascending=False)
  return df

def from_embeddings_to_labeled_df(embeddings_comments, embeddings_labels, labels):
  cos_sim = util.pytorch_cos_sim(embeddings_comments, embeddings_labels)  
  df = pd.DataFrame(cos_sim, columns=labels)
  df = df.applymap(lambda x: x.cpu().numpy())
  return df

def all_labels_above_sim_thresh(df, sim_thresh, reviewIDs, reviewType):
  df = df[df >= sim_thresh]
  df["reviewID"] = reviewIDs
  df = df.dropna(thresh=2)
  df = df.set_index("reviewID").unstack().reset_index().dropna()
  df = df.rename(columns={
      0:f'{reviewType}_score',
      'level_0':f'{reviewType}_label'
      })
  return df

def create_similarity_frame_with_multiple_labels_per_comment(embeddings_comments, embeddings_labels, labels, sim_thresh, reviewIDs, reviewType):

  df = from_embeddings_to_labeled_df(embeddings_comments, embeddings_labels, labels)
  df = all_labels_above_sim_thresh(df, sim_thresh, reviewIDs, reviewType)
  return df

def bootstrap_contradiction_counts(pros_cons_df, random_seed, number_of_iterations):
  frames = [pd.DataFrame(Counter(np.random.RandomState(seed=random_seed).permutation(pros_cons_df["cons_label"]) + " | " + np.random.RandomState(seed=random_seed).permutation(pros_cons_df["pros_label"])), index=[0]).T for i in range(number_of_iterations)]
  bootstrapped_contra_counts_df = pd.concat(frames, axis=1).fillna(0)
  return bootstrapped_contra_counts_df

def bootstrap_contradiction_counts_with_seed(pros_cons_df, number_of_iterations, random_seed):
  frames = []
  for iteration in range(number_of_iterations):
    frame = pd.DataFrame(Counter(np.random.RandomState(seed=iteration).permutation(pros_cons_df["cons_label"]) + " | " + np.random.RandomState(seed=iteration+random_seed).permutation(pros_cons_df["pros_label"])), index=[0]).T
    frames.append(frame)
  bootstrapped_contra_counts_df = pd.concat(frames, axis=1).fillna(0)
  return bootstrapped_contra_counts_df


def analyze_bootstraped_contradiction_counts(pros_cons_df, bootstrapped_contra_counts_df, zscore):
  bootstrapped_contra_df = pd.concat([bootstrapped_contra_counts_df.std(axis=1), bootstrapped_contra_counts_df.mean(axis=1)], axis=1)
  bootstrapped_contra_df.columns = ["SD", "MEAN"]
  bootstrapped_contra_df["low"] = bootstrapped_contra_df["MEAN"] - bootstrapped_contra_df["SD"] * zscore
  bootstrapped_contra_df["high"] = bootstrapped_contra_df["MEAN"] + bootstrapped_contra_df["SD"] * zscore
  return bootstrapped_contra_df

def create_p_value_dict_for_contradictions(pros_cons_df, bootstrapped_contra_df, number_of_iterations):
  contra_count_df = pros_cons_df.groupby("contradiction").count()[["reviewID"]].rename(columns={"reviewID":"COUNT"})

  p_value_high_dict = {}
  p_value_low_dict = {}
  for index, row in bootstrapped_contra_df.iterrows():
    if index in contra_count_df.index:
      occurences = contra_count_df.loc[index]["COUNT"]
    else:
      occurences = 0
    p_value_high_dict[index] = len([x for x in row.values if x >= occurences]) / number_of_iterations
    p_value_low_dict[index] = len([x for x in row.values if x <= occurences]) / number_of_iterations
  
  return p_value_high_dict, p_value_low_dict

def add_p_values_for_most_frequent_combinations(pros_cons_df, bootstrapped_contra_df, p_value_dict):
  contra_count_df = pros_cons_df.groupby("contradiction").count()[["reviewID"]].rename(columns={"reviewID":"COUNT"})
  
  frame = pd.merge(contra_count_df, bootstrapped_contra_df, left_index=True, right_index=True)
  for index, row in frame.iterrows():
    frame.loc[index, "p_value"] = p_value_dict.get(index)
  return frame

def create_df_with_contradictions(pros_df, cons_df):

  pros_cons_df = pd.merge(pros_df, cons_df, left_on="reviewID", right_on="reviewID", how="outer").dropna()
  pros_cons_df["contradiction"] = pros_cons_df["cons_label"] + " | " + pros_cons_df["pros_label"]
  return pros_cons_df

def identify_significant_contradictions(pros_cons_df, random_seed, number_of_iterations, zscore=1.96, p_value = 0.05):
  #bootstrapped_contra_counts_df = bootstrap_contradiction_counts(pros_cons_df, random_seed, number_of_iterations)

  bootstrapped_contra_counts_df = bootstrap_contradiction_counts_with_seed(pros_cons_df, number_of_iterations, random_seed)
  bootstrapped_contra_df = analyze_bootstraped_contradiction_counts(pros_cons_df, bootstrapped_contra_counts_df, zscore)
  p_value_high_dict, p_value_low_dict = create_p_value_dict_for_contradictions(pros_cons_df, bootstrapped_contra_counts_df, number_of_iterations)
  pros_cons_with_p_values_high_df = add_p_values_for_most_frequent_combinations(pros_cons_df, bootstrapped_contra_df, p_value_high_dict)
  pros_cons_with_p_values_low_df = add_p_values_for_most_frequent_combinations(pros_cons_df, bootstrapped_contra_df, p_value_low_dict)
  #high_df = pros_cons_with_p_values_df[(pros_cons_with_p_values_df["COUNT"] > pros_cons_with_p_values_df["high"])].sort_index()
  #high_df = pros_cons_with_p_values_df[(pros_cons_with_p_values_df["p_value"] < p_value)].sort_index()
  return pros_cons_with_p_values_high_df, pros_cons_with_p_values_low_df

def pair_contra_pairs_into_same_row(high_df):

  high_df = high_df.reset_index()
  high_df["pair_values"] = high_df["index"].apply(lambda x: "_".join(sorted(x.split(" | "))))

  pair_dfs = [high_df[high_df.pair_values == pair] for pair in high_df.pair_values.unique() if len(high_df[high_df.pair_values == pair]) > 1]

  dfs = []
  for pair_df in pair_dfs:
    pair_1_df = pd.DataFrame(pair_df.iloc[0]).T.reset_index(drop=True)
    pair_1_df.columns = [f'{x}_1' for x in pair_1_df.columns]

    pair_2_df = pd.DataFrame(pair_df.iloc[1]).T.reset_index(drop=True)
    pair_2_df.columns = [f'{x}_2' for x in pair_2_df.columns]

    dfs.append(pd.concat([pair_1_df, pair_2_df], axis=1))
  
  if len(dfs) > 0:
    pair_df = pd.concat(dfs)
    pair_df = pair_df[pair_df.duplicated() == False]
    pair_df = pair_df.reset_index(drop=True)
  else: 
    pair_df = pd.DataFrame()
  return pair_df

def smooth_adjusted_p_values(p_value_adjust_df):

  for index, row in p_value_adjust_df.reset_index().iterrows():
    if index > 0:
      if row.p_value_adjusted < p_value_adjust_df.loc[index - 1].p_value_adjusted:
        p_value_adjust_df.loc[index, "p_value_adjusted"] = p_value_adjust_df.loc[index - 1].p_value_adjusted

  return p_value_adjust_df

def adjust_p_values_with_holm_bonferroni(labels, labels_to_be_filtered, pros_cons_with_p_values_df, p_value):

  n_labels = len(labels) - len(labels_to_be_filtered)
  m = (n_labels)**2 - n_labels
  p_value_adjust_df = pros_cons_with_p_values_df.copy()
  p_value_adjust_df = p_value_adjust_df.sort_values("p_value")
  p_value_adjust_df = p_value_adjust_df.reset_index()
  p_value_adjust_df = p_value_adjust_df[p_value_adjust_df["index"].apply(lambda x: x.split(" | ")[0] == x.split(" | ")[1]) == False].set_index("index")
  p_value_adjust_df["rank"] = np.arange(1,len(p_value_adjust_df)+1)
  p_value_adjust_df["p_value_adjusted"] = (m + 1 - p_value_adjust_df["rank"]) * p_value_adjust_df["p_value"]
  p_value_adjust_df = p_value_adjust_df.reset_index()
  p_value_adjust_df = smooth_adjusted_p_values(p_value_adjust_df)
  
  comp_adjust_df = p_value_adjust_df[p_value_adjust_df["p_value_adjusted"] < p_value]
  
  return p_value_adjust_df, comp_adjust_df


def obtain_sector_contradictions(embeddings_pros, embeddings_cons, embeddings_labels_dict, labels, labels_to_be_filtered, reviewIDs, sim_thresh, p_value, random_seed, number_of_iterations, zscore=1.96):

  pros_df = create_similarity_frame_with_multiple_labels_per_comment(embeddings_comments = embeddings_pros, embeddings_labels = embeddings_labels_dict["embeddings_labels_pro"], labels = labels, sim_thresh=sim_thresh, reviewIDs = reviewIDs, reviewType = "pros")
  cons_df = create_similarity_frame_with_multiple_labels_per_comment(embeddings_comments = embeddings_cons, embeddings_labels = embeddings_labels_dict["embeddings_labels_con"], labels = labels, sim_thresh=sim_thresh, reviewIDs = reviewIDs, reviewType = "cons")
  pros_cons_df = create_df_with_contradictions(pros_df, cons_df)
  reviewIDs_to_exclude = pros_cons_df[(pros_cons_df.pros_label.isin(labels_to_be_filtered) == True) | (pros_cons_df.cons_label.isin(labels_to_be_filtered) == True)]["reviewID"]
  pros_cons_df = pros_cons_df[pros_cons_df.reviewID.isin(reviewIDs_to_exclude) == False]

  pros_cons_with_p_values_high_df, pros_cons_with_p_values_low_df = identify_significant_contradictions(pros_cons_df, random_seed, number_of_iterations, zscore=zscore, p_value = p_value)
  
  p_value_high_adjust_df, high_adjust_df = adjust_p_values_with_holm_bonferroni(labels, labels_to_be_filtered, pros_cons_with_p_values_high_df, p_value)
  p_value_low_adjust_df, low_adjust_df = adjust_p_values_with_holm_bonferroni(labels, labels_to_be_filtered, pros_cons_with_p_values_low_df, p_value)

  #pair_high_df = pair_contra_pairs_into_same_row(high_df)
  pair_high_adjust_df = pair_contra_pairs_into_same_row(high_adjust_df)

  #pair_low_df = pair_contra_pairs_into_same_row(high_df)
  pair_low_adjust_df = pair_contra_pairs_into_same_row(low_adjust_df)

  return pros_df, cons_df, pros_cons_df, pros_cons_with_p_values_high_df, pros_cons_with_p_values_low_df, pair_high_adjust_df, pair_low_adjust_df, p_value_high_adjust_df, p_value_low_adjust_df, high_adjust_df, low_adjust_df, pair_high_adjust_df, pair_low_adjust_df

### Tradeoffs in relation to companies

In [5]:
def complete_tradeoff_positions_count_df(tradeoff_positions_count_df, contradictions_in_companies_df, max_firms):

  for contra in list(set(tradeoff_positions_count_df.index.get_level_values(0))):
    contra_frame = contradictions_in_companies_df[contradictions_in_companies_df["sector_contradiction_1"] == contra]
    num_of_companies_with_contra = len(contra_frame)

    sub_frame_1 = pd.DataFrame({"sector_contradiction":[contra], "significant_conflict":["both"], "COUNT":[num_of_companies_with_contra]}).set_index(["sector_contradiction", "significant_conflict"])
    
    for item in list(set(tradeoff_positions_count_df.loc[contra].index)):
      
      tradeoff_positions_count_df.loc[contra, item]["COUNT"] = tradeoff_positions_count_df.loc[contra, item]["COUNT"] - num_of_companies_with_contra
    
    number_of_companies_with_trait = tradeoff_positions_count_df.loc[contra]["COUNT"].sum()

    sub_frame_2 = pd.DataFrame({"sector_contradiction":[contra], "significant_conflict":["neither nor"], "COUNT":[max_firms - number_of_companies_with_trait - 2*num_of_companies_with_contra]}).set_index(["sector_contradiction", "significant_conflict"])

    tradeoff_positions_count_df = pd.concat([tradeoff_positions_count_df, sub_frame_1, sub_frame_2])

  tradeoff_positions_count_df = tradeoff_positions_count_df.sort_index()
  return tradeoff_positions_count_df


def create_df_with_contradictions_within_companies(companies_high_df, company_names):

  company_pair_dfs = []

  for company_name in company_names:
    company_high_df = companies_high_df[companies_high_df.company_name == company_name]
    if len(company_high_df) > 0:
      company_pair_df = pair_contra_pairs_into_same_row(company_high_df)
      company_pair_dfs.append(company_pair_df)

  if len(company_pair_dfs) > 0:
    contradictions_in_companies_df = pd.concat(company_pair_dfs)
  else: contradictions_in_companies_df = pd.DataFrame()

  return contradictions_in_companies_df

def identify_conflicts_in_n_largest_firms(df, sector_contradictions, company_names, labels, labels_to_be_filtered, labels_to_be_explicitly_ignored, sim_thresh):

  company_tradeoff_pos_dfs = []

  for company_name in company_names:
    print(company_name)

    company_df = df[df["employerName"] == company_name]

    company_reviewIDs = company_df["reviewID"].to_list()
    company_pros = company_df.pros.to_list()
    company_cons = company_df.cons.to_list()

    company_embeddings_pros = model.encode(company_pros, convert_to_tensor=True, device=0)
    company_embeddings_cons = model.encode(company_cons, convert_to_tensor=True, device=0)
    #embeddings_labels = model.encode(labels,convert_to_tensor=True, device=0)
    #sim_thresh = 0.35

    company_pros_df, company_cons_df, company_pros_cons_df, company_high_df, company_pair_df = obtain_sector_contradictions(company_embeddings_pros, company_embeddings_cons, embeddings_labels, labels, labels_to_be_filtered, labels_to_be_explicitly_ignored, company_reviewIDs, sim_thresh)
    company_pair_df["company_name"] = company_name
    company_pair_df["sim_thresh"] = sim_thresh

    company_high_df = company_high_df.reset_index().rename(columns={"index":"significant_conflict"})

    pair_sides = []
    for sector_contradiction in sector_contradictions:

      pair = sector_contradiction.split("_")
      pair_side = company_high_df[(company_high_df["significant_conflict"].str.contains(pair[0])) & (company_high_df["significant_conflict"].str.contains(pair[-1]))]
      #pair_side = company_high_df[(company_high_df["significant_conflict"].isin([pair[0]])) & (company_high_df["significant_conflict"].isin([pair[-1]]))]
      pair_side["sector_contradiction"] = sector_contradiction
      pair_sides.append(pair_side)

    company_tradeoff_pos_df = pd.concat(pair_sides)
    company_tradeoff_pos_df["company_name"] = company_name

    company_tradeoff_pos_dfs.append(company_tradeoff_pos_df)

  tradeoff_positions_df = pd.concat(company_tradeoff_pos_dfs)
  return tradeoff_positions_df

def retrieve_n_largest_firms_df(df, max_firms):

  firm_review_count_df = df.groupby("employerName").count()[["reviewID"]].rename(columns={"reviewID":"review_count"})
  n_largest_firms_df = firm_review_count_df.sort_values("review_count", ascending=False).iloc[:max_firms]
  return n_largest_firms_df

def add_missing_pair(tradeoff_positions_count_df):
  sub_frames = []

  sector_contradictions = tradeoff_positions_count_df.index.get_level_values(0).unique()
  for sector_contradiction in sector_contradictions:

    sub_frame = tradeoff_positions_count_df.loc[sector_contradiction]
    if len(sub_frame) == 3:
      sub_frame = sub_frame.reset_index()
      pair = [x for x in sub_frame.significant_conflict.to_list() if "|" in x][0]
      missing_pair = f'{pair.split(" | ")[-1]} | {pair.split(" | ")[0]}'
      
      sub_frame = sub_frame.append({"significant_conflict":missing_pair, "COUNT":0}, ignore_index=True)
      sub_frame = sub_frame.reset_index(drop=True)
      sub_frame["sector_contradiction"] = sector_contradiction
      sub_frames.append(sub_frame)
    else:
      sub_frame = sub_frame.reset_index()
      sub_frame["sector_contradiction"] = sector_contradiction
      sub_frames.append(sub_frame)


  return pd.concat(sub_frames).set_index(["sector_contradiction", "significant_conflict"])
  


In [6]:
def retrieve_examples_for_conflicts_in_contradictions(pros_cons_df, df, sector_contradictions):
  templates_1 = [f'{x.split("_")[0]} | {x.split("_")[-1]}' for x in sector_contradictions]
  templates_2 = [f'{x.split("_")[-1]} | {x.split("_")[0]}' for x in sector_contradictions]
  conflicts_df = pros_cons_df[pros_cons_df.contradiction.isin(templates_1 + templates_2)]
  conflicts_df = add_examples_to_conflicts_df(conflicts_df, df)
  return conflicts_df

def retrieve_examples_for_conflicts(pros_cons_df, df, conflicts):
  conflicts_df = pros_cons_df[pros_cons_df.contradiction.isin(conflicts)]
  conflicts_df = add_examples_to_conflicts_df(conflicts_df, df)
  return conflicts_df

def add_examples_to_conflicts_df(conflicts_df, df):

  conflicts_df = pd.merge(conflicts_df, df[["reviewID", "cons", "pros", "employerName"]], left_on="reviewID", right_on="reviewID")
  conflicts_df = conflicts_df.rename(columns={"contradiction":"contradiction_side"})
  conflicts_df["contradiction"] = conflicts_df.contradiction_side.apply(lambda x: "_".join(sorted(x.split(" | "))))
  conflicts_df = conflicts_df[["contradiction", "contradiction_side", "cons_label", "pros_label", "cons", "pros", "cons_score", "pros_score", "employerName", "reviewID"]]
  conflicts_df = conflicts_df[conflicts_df.duplicated() == False]
  conflicts_df["combined_score"] = (conflicts_df["cons_score"] + conflicts_df["pros_score"]) / 2 - abs(conflicts_df["cons_score"] - conflicts_df["pros_score"])

  return conflicts_df.sort_values(["cons_label", "pros_label"], ascending=False)
  #return conflicts_df.sort_values(["cons_score", "pros_label"], ascending=False)

def add_missing_contradiction_sides(companies_contradiction_df):

  company_contra_dfs = []

  company_names = companies_contradiction_df.employerName.unique()
  for company_name in company_names:
    company_df = companies_contradiction_df[companies_contradiction_df.employerName == company_name]
    company_contradictions = company_df.contradiction.unique()
    for company_contradiction in company_contradictions:
      company_contra_df = company_df[company_df.contradiction == company_contradiction]
      
      if len(company_contra_df) == 1:
        contradiction_side = company_contra_df.contradiction_side.iloc[0]
        contra_count = company_contra_df.contra_count.iloc[0]
        
        missing_contradiction_side = f'{contradiction_side.split(" | ")[-1]} | {contradiction_side.split(" | ")[0]}'
        company_contra_df = company_contra_df.append({
            "employerName":company_name, 
            "contradiction":company_contradiction, 
            "contradiction_side":missing_contradiction_side, 
            "contra_side_count":0,
            "contra_count":contra_count
            }, ignore_index=True)
        company_contra_df = company_contra_df.reset_index(drop=True)
        
      company_contra_dfs.append(company_contra_df)

  return pd.concat(company_contra_dfs)


def retain_only_one_side_of_contradictions(companies_contradiction_df):

  selected_sides = []

  company_contradictions = companies_contradiction_df.contradiction.unique()
  for company_contradiction in company_contradictions:
    company_contradiction_sides_df = companies_contradiction_df[companies_contradiction_df.contradiction == company_contradiction]
    company_contradiction_sides = list(company_contradiction_sides_df.contradiction_side.unique())
    selected_sides.append(sorted(company_contradiction_sides)[0])
  return selected_sides

def filter_by_minimal_occurence(companies_contradiction_df, min_n_item):
  #companies_contradiction_df = companies_contradiction_df[(companies_contradiction_df.contra_side_count + companies_contradiction_df.contra_count) >= min_n_item]
  companies_contradiction_df = companies_contradiction_df[companies_contradiction_df.contra_count >= min_n_item]
  return companies_contradiction_df

def ratio_into_bin(number, bin_dict):
  for key in bin_dict.keys():
    if number <= key:
      return bin_dict.get(key)
      break


### pro-pro and con-con pairs

In [7]:
def get_review_ids_with_mulptiple_labels_in_column(df, column_type):
  """
  df: [pros_df, cons_df]
  column_type: ["pros", "cons"]
  """

  #pros_pairs_df = pros_df.groupby(["reviewID", f"{column_type}_label"]).count().rename(columns={f"{column_type}_score":"COUNT"}).sort_values("COUNT", ascending=False)
  #pros_pairs_df = pros_pairs_df[pros_pairs_df["COUNT"] > 1]

  review_ids_with_multiple_labels_df = df.groupby(["reviewID"]).count().rename(columns={f"{column_type}_score":"COUNT"})[["COUNT"]]
  review_ids_with_multiple_labels = review_ids_with_multiple_labels_df[review_ids_with_multiple_labels_df["COUNT"] > 1].index.to_list()

  return review_ids_with_multiple_labels


# identify_pairs_in_either_pros_or_cons()

#review_ids_with_multiple_labels

def identify_pairs_in_either_pros_or_cons_column(df, column_type, labels_to_be_filtered):
  """
  df: [pros_df, cons_df]
  column_type: ["pros", "cons"]
  """

  review_ids_with_multiple_labels = get_review_ids_with_mulptiple_labels_in_column(df, column_type)
  frame = df[(df.reviewID.isin(review_ids_with_multiple_labels)) & (df[f"{column_type}_label"].isin(labels_to_be_filtered) == False)].sort_values("reviewID")
  reviewIDs = frame.reviewID.unique()

  pairs_list = []
  for reviewID in reviewIDs:
    label_list = frame[frame.reviewID == reviewID][f"{column_type}_label"].to_list()
    pairs = list(set([l for sublist in [["_".join(sorted([label, x])) for x in label_list if x != label] for label in label_list] for l in sublist]))
    pairs_list.append(pairs)

  pair_count_df = pd.DataFrame(Counter([l for sublist in pairs_list for l in sublist]), index=["COUNT"]).T.sort_values("COUNT", ascending=False)[["COUNT"]].reset_index().rename(columns={"index":"pair"})
  pair_count_dict = pair_count_df.set_index("pair")["COUNT"].to_dict()
  return pair_count_df, frame, pair_count_dict

def add_missing_bins(frame):
  for ratio_bin in [x for x in range(1,7) if x not in frame.ratio_bin.to_list()]:
    frame = frame.append({"ratio_bin":ratio_bin, "bin_count":0}, ignore_index=True)
  frame["ratio_bin"] = frame.ratio_bin.astype(int)
  frame = frame.sort_values("ratio_bin")
  return frame

def bootstrap_random_counts_per_pair(df, column_type, number_of_iterations):
  """
  df: [pros_frame, cons_frame]
  column_type: ["pros", "cons"]
  """
  frames = []
  for iteration in range(number_of_iterations):
    frames.append(pd.DataFrame(Counter(df[f"{column_type}_label"] + "_" + np.random.RandomState(seed=iteration).permutation(df[f"{column_type}_label"])), index=[0]).T)

  bootstrapped_counts_df = pd.concat(frames, axis=1)
  bootstrapped_counts_df = bootstrapped_counts_df.fillna(0)
  bootstrapped_counts_df = bootstrapped_counts_df.reset_index()

  bootstrapped_counts_df["pair"] = bootstrapped_counts_df["index"].apply(lambda x: "_".join(sorted(x.split("_"))))
  bootstrapped_counts_df = bootstrapped_counts_df.groupby("pair").sum()
  return bootstrapped_counts_df



def summarize_random_counts_metrics_for_pairs(bootstrapped_counts_df, pair_counts_dict):
  for pair, row in bootstrapped_counts_df.iterrows():
    if pair in pair_counts_dict.keys():
      orig_count = pair_counts_dict.get(pair)

      bootstrapped_counts_df.loc[pair, "mean_bootstrapped"] = row.mean()
      bootstrapped_counts_df.loc[pair, "std_bootstrapped"] = row.std()
      bootstrapped_counts_df.loc[pair, "upper_limit_bootstrapped"] = row.mean() + (z_score * row.std())
      bootstrapped_counts_df.loc[pair, "lower_limit_bootstrapped"] = row.mean() - (z_score * row.std())
      bootstrapped_counts_df.loc[pair, "p_value_high"] = len(row[row >= orig_count]) / number_of_iterations
      bootstrapped_counts_df.loc[pair, "p_value_low"] = len(row[row <= orig_count]) / number_of_iterations

  return bootstrapped_counts_df

def identify_frequent_and_infrequent_pairs_in_column(bootstrapped_counts_df, pairs_df):
  """
  pairs_df: [pros_pairs_df, cons_pairs_df]
  """

  bootstrapped_counts_df_high = bootstrapped_counts_df[bootstrapped_counts_df.p_value_high == 0][["mean_bootstrapped", "std_bootstrapped", "upper_limit_bootstrapped", "lower_limit_bootstrapped", "p_value_high", "p_value_low"]]
  bootstrapped_counts_df_low = bootstrapped_counts_df[bootstrapped_counts_df.p_value_low == 0][["mean_bootstrapped", "std_bootstrapped", "upper_limit_bootstrapped", "lower_limit_bootstrapped", "p_value_high", "p_value_low"]]
  frequent_pairs_df = pd.merge(pairs_df, bootstrapped_counts_df_high, left_on="pair", right_index=True)
  infrequent_pairs_df = pd.merge(pairs_df, bootstrapped_counts_df_low, left_on="pair", right_index=True)
  

  return frequent_pairs_df, infrequent_pairs_df

def identify_frequent_and_infrequent_pairs_by_column(df, pairs_df, column_type, pair_counts_dict, number_of_iterations):

  """
  df: [pros_frame, cons_frame]
  column_type: ["pros", "cons"]
  """

  bootstrapped_counts_df = bootstrap_random_counts_per_pair(df, column_type, number_of_iterations)
  bootstrapped_counts_df = summarize_random_counts_metrics_for_pairs(bootstrapped_counts_df, pair_counts_dict)
  frequent_pairs_df, infrequent_pairs_df = identify_frequent_and_infrequent_pairs_in_column(bootstrapped_counts_df, pairs_df)
  frequent_pairs_df[f'{column_type}_pair'] = "frequent"
  infrequent_pairs_df[f'{column_type}_pair'] = "infrequent"

  return frequent_pairs_df, infrequent_pairs_df

def create_summary_of_summary_of_significant_pairings_in_same_column(pros_frame, pros_pairs_df, cons_frame, cons_pairs_df, pros_pair_count_dict, cons_pair_count_dict, number_of_iterations):

  pros_frequent_pairs_df, pros_infrequent_pairs_df = identify_frequent_and_infrequent_pairs_by_column(pros_frame, pros_pairs_df, "pros", pros_pair_count_dict, number_of_iterations)
  cons_frequent_pairs_df, cons_infrequent_pairs_df = identify_frequent_and_infrequent_pairs_by_column(cons_frame, cons_pairs_df, "cons", cons_pair_count_dict, number_of_iterations)
  summary_of_significant_pairings_in_same_column_df = pd.concat([cons_frequent_pairs_df, cons_infrequent_pairs_df, pros_frequent_pairs_df, pros_infrequent_pairs_df])

  #summary_of_significant_pairings_in_same_column_df["cons_pair"] = summary_of_significant_pairings_in_same_column_df["cons_pair"].fillna("insignificant")
  #summary_of_significant_pairings_in_same_column_df["pros_pair"] = summary_of_significant_pairings_in_same_column_df["pros_pair"].fillna("insignificant")
  
  return summary_of_significant_pairings_in_same_column_df

def create_dict_for_pairing_frequency_classes(summary_of_significant_pairings_in_same_column_df):

  pairing_dict = {}

  for pair in summary_of_significant_pairings_in_same_column_df.pair.unique():
    pairing_dict[pair] = {}
    sub_frame = summary_of_significant_pairings_in_same_column_df[summary_of_significant_pairings_in_same_column_df.pair == pair]
    
    result = sub_frame["cons_pair"].dropna().to_list()
    if len(result) == 0: result = "insignificant"
    else: result = result[0]

    pairing_dict[pair]["cons_pair"] = result

    result = sub_frame["pros_pair"].dropna().to_list()
    if len(result) == 0: result = "insignificant"
    else: result = result[0]

    pairing_dict[pair]["pros_pair"] = result

  return pairing_dict

### Prepare data for export

In [8]:
def clean_df_with_conflict_overview(p_value_adjust_df):
  df_cleaned = p_value_adjust_df.copy().reset_index().rename(columns={"index":"pair"})
  df_cleaned = df_cleaned[df_cleaned.p_value_adjusted < p_value]

  # remove rows with pairs of equal labels
  df_cleaned = df_cleaned[df_cleaned.pair.apply(lambda x: x.split(" | ")[0] != x.split(" | ")[-1])]
  df_cleaned["conflict"] = df_cleaned["pair"].apply(lambda x: "_".join(sorted(x.split(" | "))))
  df_cleaned = df_cleaned.sort_values("conflict")
  #high_df_cleaned = high_df_cleaned.set_index(["tradeoff", "pair"])
  return df_cleaned



def get_tradeoff_and_non_tradeoff_pairs(df_cleaned):
  tradeoffs = [pair for pair in high_pairs if len(df_cleaned[df_cleaned["conflict"] == pair]) == 2]
  non_tradeoff_pairs = [pair for pair in high_pairs if len(df_cleaned[df_cleaned["conflict"] == pair]) != 2]
  return tradeoffs, non_tradeoff_pairs



def create_final_tradeoff_df(df_cleaned, list_of_items):

  tradeoff_df = df_cleaned[df_cleaned["conflict"].isin(list_of_items)] #.set_index(["tradeoff", "pair"])
  tradeoff_df["con"] = tradeoff_df["pair"].apply(lambda x: x.split(" | ")[0])
  tradeoff_df["pro"] = tradeoff_df["pair"].apply(lambda x: x.split(" | ")[-1])
  tradeoff_df["tradeoff"] = tradeoff_df["conflict"].apply(lambda x: x.replace("_", " vs. "))
  tradeoff_df = tradeoff_df.set_index(["tradeoff", "con", "pro"]).drop("pair", axis=1).rename(columns = {"COUNT":"count", "p_value":"p-value", "p_value_adjusted":"p-value adjusted"})
  return tradeoff_df



def create_final_non_tradeoff_df(df_cleaned, list_of_items):

  non_tradeoff_df = df_cleaned[df_cleaned["conflict"].isin(list_of_items)] #
  non_tradeoff_df["con"] = non_tradeoff_df["pair"].apply(lambda x: x.split(" | ")[0])
  non_tradeoff_df["pro"] = non_tradeoff_df["pair"].apply(lambda x: x.split(" | ")[-1])
  non_tradeoff_df = non_tradeoff_df.sort_values(["con", "pro"]).reset_index(drop=True).drop("pair", axis=1) #.drop("conflict", axis=1)
  
  non_tradeoff_df = non_tradeoff_df[["con", "pro", "COUNT", "p_value", "conflict"]].rename(columns = {"COUNT":"count", "p_value":"p-value", "p_value_adjusted":"p-value adjusted"}).set_index(["con", "pro"])
  return non_tradeoff_df

def retrieve_good_examples(conflicts_df, n_of_examples):

  frames = []

  for tradeoff in conflicts_df.contradiction.unique():
    frame = conflicts_df[conflicts_df.contradiction == tradeoff]
    for contradiction_side in frame.contradiction_side.unique():
      sub_frame = conflicts_df[conflicts_df.contradiction_side == contradiction_side]
      frames.append(sub_frame.sort_values("combined_score", ascending=False).iloc[:n_of_examples])

  good_examples_df = pd.concat(frames)
  good_examples_df["tradeoff"] = good_examples_df.contradiction.apply(lambda x: x.replace("_", " vs. "))
  good_examples_df = good_examples_df.set_index(["tradeoff", "cons_label", "pros_label", "contradiction"])

  return good_examples_df

def add_in_column_frequency_class_to_tradeoff_dfs(df, pairing_dict):

  df["cons pair"] = df["conflict"].apply(lambda x: pairing_dict.get(x)['cons_pair'] if x in pairing_dict.keys() else "insignificant")
  df["pros pair"] = df["conflict"].apply(lambda x: pairing_dict.get(x)['pros_pair'] if x in pairing_dict.keys() else "insignificant")
  return df



## Load data

Note that you will need access to Glassdoor's proprietary data to run the code. If you just want to test with a dummy frame, you will need the following columns:

| reviewID | pros | cons | employerName |
| --- | --- | --- | --- |
| some number | some string | another string | another string |


In [ ]:
sector_name = "Finance"
#sector_name = "Manufacturing"
df = pd.read_csv(f"/content/{sector_name}.csv")
print(len(df))

## Execution

### Identify the labels (i.e., the categories that will serve as topics)

In [ ]:
random_seed = 37
number_of_sampled_comments = 4000
number_of_most_frequent_words = 15
dissimilarity_threshold = 0.3
stopwords_to_be_added = ["lot","con","pro", "thing"]

labels_to_be_explicitly_ignored = ["place", '-pron-', 'great']
labels, noun_df = get_most_representative_labels(df, number_of_sampled_comments, number_of_most_frequent_words, labels_to_be_explicitly_ignored, dissimilarity_threshold, stopwords_to_be_added, random_seed, use_similarity_reduction=True)
labels_to_be_filtered = ["nothing negative to say", "nothing positive to say", "none"]
labels = labels + labels_to_be_filtered
labels = [x for x in labels if x not in labels_to_be_explicitly_ignored]

### Get the sentence embeddings for the pros and cons sections

In [ ]:

df = pd.read_csv(f"/content/{sector_name}.csv")

reviewIDs = df["reviewID"].to_list()
pros = df.pros.to_list()
cons = df.cons.to_list()

pros_prefix = ""
cons_prefix = ""

embeddings_pros = model.encode([f'{pros_prefix} {x}'.strip() for x in pros],convert_to_tensor=True, device=0)
embeddings_cons = model.encode([f'{cons_prefix} {x}'.strip() for x in cons],convert_to_tensor=True, device=0)


### Get the sentence embeddings for the labels

In [ ]:
pro_labels = [f'good {x}' for x in labels]
con_labels = [f'bad {x}' for x in labels]
embeddings_labels_pro = model.encode(pro_labels,convert_to_tensor=True, device=0)
embeddings_labels_con = model.encode(con_labels,convert_to_tensor=True, device=0)
embeddings_labels = model.encode(labels,convert_to_tensor=True, device=0)

### Identify pro-con pairs

In [ ]:
sim_thresh = 0.35
minimal_n_of_pairings = 0
number_of_iterations = 10000
p_value = 0.05

embeddings_labels_dict = {
    "embeddings_labels_pro":embeddings_labels,
    "embeddings_labels_con":embeddings_labels,
}

pros_df, cons_df, pros_cons_df, pros_cons_with_p_values_high_df, pros_cons_with_p_values_low_df, pair_high_adjust_df, pair_low_adjust_df, p_value_high_adjust_df, p_value_low_adjust_df, high_adjust_df, low_adjust_df, pair_high_adjust_df, pair_low_adjust_df = obtain_sector_contradictions(embeddings_pros, embeddings_cons, embeddings_labels_dict, labels, labels_to_be_filtered, reviewIDs, sim_thresh, p_value, random_seed, number_of_iterations)

In [ ]:
pair_high_adjust_df["sector"] = sector_name
pair_high_adjust_df["sim_thresh"] = sim_thresh

pair_low_adjust_df["sector"] = sector_name
pair_low_adjust_df["sim_thresh"] = sim_thresh

sector_contradictions = pair_high_adjust_df.pair_values_1.to_list()

In [ ]:


contras_examples_df = retrieve_examples_for_conflicts_in_contradictions(pros_cons_df, df, sector_contradictions)
infrequent_examples_df = retrieve_examples_for_conflicts(pros_cons_df, df, low_adjust_df["index"].to_list())
frequent_examples_df = retrieve_examples_for_conflicts(pros_cons_df, df, high_adjust_df["index"].to_list())

In [ ]:
companies_contradiction_side_count_df_1 = contras_examples_df.groupby(["employerName", "contradiction", "contradiction_side"]).count()[["reviewID"]].rename(columns={"reviewID":"contra_side_count"})
companies_contradiction_side_count_df_2 = contras_examples_df.groupby(["employerName", "contradiction"]).count()[["reviewID"]].rename(columns={"reviewID":"contra_count"})
companies_contradiction_df = pd.merge(companies_contradiction_side_count_df_1, companies_contradiction_side_count_df_2, left_index=True, right_index=True)
companies_contradiction_df = companies_contradiction_df.reset_index()
companies_contradiction_df = add_missing_contradiction_sides(companies_contradiction_df)
companies_contradiction_df["ratio"] = companies_contradiction_df["contra_side_count"] / companies_contradiction_df["contra_count"]
selected_sides = retain_only_one_side_of_contradictions(companies_contradiction_df)
companies_contradiction_df = companies_contradiction_df[companies_contradiction_df.contradiction_side.isin(selected_sides)]


In [ ]:
n_of_bins = 6
bin_dict = {1/n_of_bins + x/n_of_bins : i+1 for i, x in enumerate(range(n_of_bins))}

companies_contradiction_df["ratio_bin"] = companies_contradiction_df.ratio.apply(lambda x: ratio_into_bin(x, bin_dict))
frame_1 = filter_by_minimal_occurence(companies_contradiction_df, 1)
frame_2 = filter_by_minimal_occurence(companies_contradiction_df, 2)

frame_1_bin_counts_df = frame_1.groupby(["contradiction", "contradiction_side", "ratio_bin"]).count()[["employerName"]].rename(columns={"employerName":"bin_count"}).reset_index()
frame_2_bin_counts_df = frame_2.groupby(["contradiction", "contradiction_side", "ratio_bin"]).count()[["employerName"]].rename(columns={"employerName":"bin_count"}).reset_index()

In [ ]:
number_of_columns = 2

contradiction_sides = list(companies_contradiction_df.contradiction_side.unique())
number_of_sector_contradictions = len(contradiction_sides)
number_of_rows = int(np.ceil(number_of_sector_contradictions / number_of_columns))

fig = make_subplots(rows=number_of_rows, cols=number_of_columns, vertical_spacing = 0.15, horizontal_spacing = 0.15,
                    subplot_titles=[x.replace("|", "vs.") for x in contradiction_sides])

contra_counter = 0

for i, number_of_row in enumerate(range(1,number_of_rows+1)):
  for j, number_of_column in enumerate(range(1,number_of_columns+1)):

    if contra_counter < number_of_sector_contradictions:
      contradiction_side = contradiction_sides[contra_counter]
      contradiction_side_label = f'bad: {contradiction_side.split(" | ")[0]}<br>good: {contradiction_side.split(" | ")[-1]}'
      contradiction_side_opposite = f'bad: {contradiction_side.split(" | ")[-1]}<br>good: {contradiction_side.split(" | ")[0]}'
      
      frame = frame_1_bin_counts_df[frame_1_bin_counts_df.contradiction_side == contradiction_side]
      frame = add_missing_bins(frame)

      if (i == 0) & (j == 0): showlegend = True
      else: showlegend = False

      fig.add_trace(
          go.Bar(
              x=frame["ratio_bin"], 
              y=frame["bin_count"], 
              marker=dict(color="blue", opacity=0.5),
              showlegend=showlegend,
              name="at least 1 instance"
              ),
              row=number_of_row, col=number_of_column,
              
      )

      frame = frame_2_bin_counts_df[frame_2_bin_counts_df.contradiction_side == contradiction_side]
      frame = add_missing_bins(frame)

      fig.add_trace(
          go.Bar(
              x=frame["ratio_bin"], 
              y=frame["bin_count"], 
              marker=dict(color="red", opacity=0.5),
              showlegend=showlegend,
              name="at least 2 instances"
              ),
              row=number_of_row, col=number_of_column,
              
      )

      fig.update_yaxes(
          title_text="# of companies", 
          gridcolor="grey",
          type="log",
          row=number_of_row, 
          col=number_of_column)
      
      fig.update_xaxes(
          #title_text="yaxis 1 title", 
          ticktext=[f"<br>100 - 83%<br><br>{contradiction_side_opposite}",
                    f"<br>50%<br><br>both combinations",
                    f"<br>83 - 100%<br><br>{contradiction_side_label}"],
          tickvals=[1, 3.5, 6],
          range=[0, 7],
          row=number_of_row, 
          col=number_of_column)

    contra_counter = contra_counter + 1


fig.update_layout(
    height=number_of_rows*400, 
    width=number_of_columns*550,
    barmode="overlay",
    #title=dict(text=f"Tradeoffs in {sector_name} from the perspective of employees", x=0.5, xanchor="center", yanchor="top", y=0.99),
    margin=dict(
        t=100, 
        b=150
        ),
    legend=dict(orientation="h", xanchor = "left", x = 0, y= -0.7),
    plot_bgcolor = "rgba(0,0,0,0)")


fig.show()
fig.write_image(f'{sector_name}_figure.png', engine="kaleido", scale=2)
fig.write_image(f'{sector_name}_figure.svg', engine="kaleido")


In [ ]:
pros_pairs_df, pros_frame, pros_pair_count_dict = identify_pairs_in_either_pros_or_cons_column(pros_df, "pros", labels_to_be_filtered)
cons_pairs_df, cons_frame, cons_pair_count_dict = identify_pairs_in_either_pros_or_cons_column(cons_df, "cons", labels_to_be_filtered)

In [ ]:
pros_frame["random_label"] = np.random.permutation(pros_frame["pros_label"])
pros_frame["random_pair"] = sorted(pros_frame["random_label"] + "_" + pros_frame["pros_label"])

In [ ]:
z_score = 1.96
summary_of_significant_pairings_in_same_column_df = create_summary_of_summary_of_significant_pairings_in_same_column(pros_frame, pros_pairs_df, cons_frame, cons_pairs_df, pros_pair_count_dict, cons_pair_count_dict, number_of_iterations)
pairing_dict = create_dict_for_pairing_frequency_classes(summary_of_significant_pairings_in_same_column_df)

In [ ]:
grouped_comp_contra_df = frame_2.set_index(["contradiction", "employerName"])
contradictions = grouped_comp_contra_df.index.get_level_values(0).unique()

frames = []

for contradiction in contradictions:

  frames.append(grouped_comp_contra_df.loc[contradiction][["ratio"]].rename(columns={"ratio":contradiction}))

merged_df = frames[0]
for frame in frames[1:]:
  merged_df = pd.merge(merged_df, frame, left_index=True, right_index=True, how="outer")
corr_df = merged_df.corr()

In [ ]:
mask = np.triu(np.ones_like(corr_df, dtype=bool))
rLT = corr_df.applymap(lambda x: np.abs(x)).mask(mask)

fig = go.Figure(data=go.Heatmap(
                   z=rLT,
                   x=[x.replace("_", " vs. ") for x in rLT.columns],
                   y=[x.replace("_", " vs. ") for x in rLT.columns],
                   hoverongaps = False,
                   #colorscale = 'RdBu',
                   ))

fig.update_xaxes(tickangle=90, showgrid=False)
fig.update_yaxes(showgrid=False)


margin = 250
fig.update_layout(
    width=600,
    height=600,
    plot_bgcolor="rgba(0,0,0,0)",
    margin=dict(pad=20, b=margin, l=200),
    yaxis_autorange='reversed',
    
    
)
fig.show()

fig.write_image(f'{sector_name}_figure_corr_matrix.png', engine="kaleido", scale=3)

In [ ]:
labels_used = [x for x in labels if x not in labels_to_be_filtered + labels_to_be_explicitly_ignored]
labels_used

In [ ]:
high_df_cleaned = clean_df_with_conflict_overview(p_value_high_adjust_df)
low_df_cleaned = clean_df_with_conflict_overview(p_value_low_adjust_df)

high_pairs = high_df_cleaned["conflict"].unique()



In [ ]:
tradeoff_high_df = create_final_tradeoff_df(high_df_cleaned, tradeoffs_high)
tradeoff_low_df = create_final_tradeoff_df(low_df_cleaned, tradeoffs_low)

non_tradeoff_high_df = create_final_non_tradeoff_df(high_df_cleaned, non_tradeoff_pairs_high)
non_tradeoff_low_df = create_final_non_tradeoff_df(low_df_cleaned, low_df_cleaned.conflict.to_list())

tradeoffs_high, non_tradeoff_pairs_high = get_tradeoff_and_non_tradeoff_pairs(high_df_cleaned)
tradeoffs_low, non_tradeoff_pairs_low = get_tradeoff_and_non_tradeoff_pairs(low_df_cleaned)

In [ ]:
contras_good_examples_df = retrieve_good_examples(contras_examples_df, 3)
infrequent_good_examples_df = retrieve_good_examples(infrequent_examples_df, 3)
frequent_good_examples_df = retrieve_good_examples(frequent_examples_df, 3)

In [ ]:
non_tradeoff_high_df = add_in_column_frequency_class_to_tradeoff_dfs(non_tradeoff_high_df, pairing_dict)
tradeoff_high_df = add_in_column_frequency_class_to_tradeoff_dfs(tradeoff_high_df, pairing_dict)
non_tradeoff_low_df = add_in_column_frequency_class_to_tradeoff_dfs(non_tradeoff_low_df, pairing_dict)
tradeoff_low_df = add_in_column_frequency_class_to_tradeoff_dfs(tradeoff_low_df, pairing_dict)

### Output of summary file

In [ ]:
path = "/content"

with pd.ExcelWriter(f"{path}/{sector_name}_{str(sim_thresh).replace('.', 'dot')}_summary.xlsx", engine='xlsxwriter') as writer:
  tradeoff_high_df.to_excel(writer, "tradeoffs")
  non_tradeoff_high_df.to_excel(writer, "non-tradeoff pairs")

  if len(tradeoff_low_df) > 0: tradeoff_low_df.to_excel(writer, "infreq tradeoffs")
  non_tradeoff_low_df.to_excel(writer, "infreq non-tradeoff pairs")
  
  contras_good_examples_df.to_excel(writer, "examples contras")
  frequent_good_examples_df.to_excel(writer, "examples frequent")
  infrequent_good_examples_df.to_excel(writer, "examples infrequent")
  
  contras_examples_df.to_excel(writer, "all tradeoff comms")
  infrequent_examples_df.to_excel(writer, "all infrequent comms")
  frequent_examples_df.to_excel(writer, "all frequent comms")
  pd.DataFrame().to_excel(writer, "figures")

  summary_of_significant_pairings_in_same_column_df.to_excel(writer, "pair frequencies")
  pd.DataFrame({"labels":labels_used}).to_excel(writer, "list of labels")

  workbook  = writer.book
  worksheet = writer.sheets['figures']

  worksheet.insert_image('D3', f'{sector_name}_figure.png')
  worksheet.insert_image('D20', f'{sector_name}_figure_corr_matrix.png')